## Preparations

In [1]:
# !auto-py-to-exe

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
print("d2loadout update started")

d2loadout update started


In [4]:
import json
import os
import re
import pandas as pd
import numpy as np
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import ast
import html_to_json

In [5]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("user-agent=" + UserAgent().random)
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("detach", True)

# chrome_options.add_argument('headless')
# chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument("disable-gpu")

chrome_options.page_load_strategy = 'eager'


os.environ["PATH"] += r"C:\Program Files (x86)\Chromedriver.exe"
driver = webdriver.Chrome(options=chrome_options)

## Meta

In [6]:
link_meta = "https://dota2protracker.com/meta"
driver.implicitly_wait(3)
driver.get(link_meta)
driver.implicitly_wait(10)

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time

def get_d2pt_page_table(driver):
    # Даем время на загрузку контента
    time.sleep(0.2)

    # Получаем HTML страницы
    page_source = driver.page_source

    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Ищем строки таблицы по их CSS-классу
    table_rows = soup.find_all('div', class_='grid', style=True)

    # Список для хранения данных
    data = []
    headers = []

    for index, row in enumerate(table_rows):
        # Проверяем, чтобы строка не была заголовком
        cols = row.find_all('div', recursive=False)

        if index == 0:
            # Считаем первую строку заголовком
            for col in cols:
                header_text = col.get_text(strip=True)
                headers.append(header_text if header_text else None)
            continue

        row_data = []
        for col in cols:
            # Проверяем наличие изображений для извлечения роли
            img = col.find('img', alt=True)
            if img:
                row_data.append(img['alt'])
                continue

            # Обрабатываем содержимое внутри span
            spans = col.find_all('span')
            if spans:
                span_text = ' '.join([span.get_text(strip=True) for span in spans])
                row_data.append(span_text if span_text else None)
                continue

            # Извлекаем текст и убираем пустые значения
            text = col.get_text(strip=True)
            row_data.append(text if text else None)

        if row_data:  # Только если данные не пусты
            data.append(row_data)

    # Преобразуем список в DataFrame
    df_heroes_table = pd.DataFrame(data, columns=headers)

    # Убираем лишние строки или обрабатываем некорректные данные
    df_heroes_table = df_heroes_table.dropna(how='all')

    # Очистка данных в процентах и диапазонах
    def clean_data(value):
        if isinstance(value, str):
            if '%' in value:
                try:
                    return float(value.replace('%', ''))
                except ValueError:
                    return value  # Вернуть исходное значение, если не удалось преобразовать
            elif '(' in value and ')' in value:
                try:
                    main_value, range_values = value.split('(')
                    main_value = main_value.strip()
                    range_values = range_values.replace(')', '').split('-')
                    return {
                        'main': int(main_value),
                        'range_min': int(range_values[0]),
                        'range_max': int(range_values[1])
                    }
                except ValueError:
                    return value  # Вернуть исходное значение, если формат не соответствует
        return value

    # Применяем очистку данных ко всем ячейкам
    df_heroes_table = df_heroes_table.applymap(clean_data)

    # Обработка типов данных для чисел
    df_heroes_table = df_heroes_table.apply(lambda x: pd.to_numeric(x, errors='ignore') if x.name != 2 else x)

    return df_heroes_table

# Пример использования:
# driver = ...  # Selenium WebDriver
# df = get_d2pt_page_table(driver)
# print(df)


In [8]:
print("checking meta...")
print("fetching heroes info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table(driver)

df_full = pd.concat([df_1, df_2, df_3, df_4, df_5], axis=0)
df_full

checking meta...
fetching heroes info for pos 1
fetching heroes info for pos 2
fetching heroes info for pos 3
fetching heroes info for pos 4
fetching heroes info for pos 5


,Rank,Role,Hero,D2PT Rating,Matches,Win Rate,WR 9500+ MMR,Contest Rate,Radiant,Dire,Expert,1st Phase,2nd Phase,Lastpick,Networth@10
0,#1,pos 1,Alchemist,3479,4813,52.8,54.4% 2284 +1.6%,34.2,53.4,52.2,58.7% (825),49.8,54.1,51.2,"{'main': 5544, 'range_min': 4633, 'range_max':..."
1,#2,pos 1,Medusa,3337,4864,52.4,51.7% 1945 -0.7%,47.3,54.8,49.9,59.4% (623),48.0,52.2,52.6,"{'main': 4577, 'range_min': 3892, 'range_max':..."
2,#3,pos 1,Shadow Fiend,3207,3953,51.6,52.9% 1313 +1.2%,72.5,54.0,49.1,56.9% (195),51.3,53.2,53.7,"{'main': 4713, 'range_min': 3864, 'range_max':..."
3,#4,pos 1,Luna,3136,6284,51.1,48.6% 2336 -2.5%,67.1,53.3,48.9,60.1% (634),51.7,50.8,58.7,"{'main': 4466, 'range_min': 3701, 'range_max':..."
4,#5,pos 1,Morphling,3095,5204,51.4,51.3% 2158 -0.1%,40.9,53.4,49.4,54.7% (1164),47.4,51.9,53.2,"{'main': 4468, 'range_min': 3764, 'range_max':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,#122,pos 5,Underlord,0,22,50.0,100.0% 6 +50.0%,5.8,40.0,52.9,50.0% (16),47.4,0.0,0.0,"{'main': 3629, 'range_min': 2540, 'range_max':..."
122,#123,pos 5,Pangolier,0,29,44.8,50.0% 10 +5.2%,28.3,50.0,36.4,0.0% (0),25.0,54.5,0.0,"{'main': 2395, 'range_min': 1707, 'range_max':..."
123,#124,pos 5,Mars,0,16,43.8,50.0% 4 +6.3%,22.7,50.0,40.0,0.0% (0),33.3,80.0,33.3,"{'main': 2302, 'range_min': 1387, 'range_max':..."
124,#125,pos 5,Void Spirit,0,7,14.3,0.0% 2 -14.3%,7.3,0.0,50.0,0.0% (0),0.0,100.0,0.0,"{'main': 2668, 'range_min': 1411, 'range_max':..."


## Facets

In [9]:
link = "https://dota2protracker.com/facets"

In [10]:
driver.implicitly_wait(2)
driver.get(link)

In [11]:
hero_req = requests.get("https://dota2protracker.com/hero/Tiny")

In [12]:
import re

# Пример HTML-кода (замени на свой)
html_text = hero_req.text
# Регулярное выражение для нахождения объекта data
pattern = r'const data = (\[.+?\]);'
constdata = re.findall(pattern, html_text, re.DOTALL)[0]


In [13]:
import js2py
import json
import pandas as pd

# Пример JavaScript-кода
js_code = f"""
var data = {constdata};

// Возвращаем JSON-строку facetData
JSON.stringify(data[2].data.facetData);
"""

# Исполняем JavaScript-код
result = js2py.eval_js(js_code)

if result:
    # Преобразуем JSON-строку в Python-словарь
    facet_data = json.loads(result)
    
    # Инициализируем пустой DataFrame
    all_facets_df = pd.DataFrame()

    # Итерация по всем ключам в facetData
    for key, value in facet_data.items():
        facets = value.get("facets", [])
        # Преобразуем каждый набор facets в DataFrame
        facets_data = []
        for facet in facets:
            if not facet.get("deprecated", False):  # Пропускаем deprecated
                facets_data.append(facet)
        
        facets_df = pd.DataFrame(facets_data)
        
        # Добавляем информацию о ключе, id и localized_name в DataFrame
        facets_df["key"] = key  # Ключ из facetData
        facets_df["id"] = value.get("id")
        facets_df["localized_name"] = value.get("localized_name")
        
        # Объединяем текущий DataFrame с общим
        all_facets_df = pd.concat([all_facets_df, facets_df], ignore_index=True)
    
    # Перенумерация facet_number
    all_facets_df["facet_number"] = (
        all_facets_df.groupby(["key", "id"]).cumcount() + 1
    )
    
    # Выводим общий DataFrame
    display(all_facets_df)
else:
    print("Данные не найдены или невалидны.")


,background,deprecated,description,icon,name,key,id,localized_name,num_matches,pickRate,winRate,facet_number
0,"linear-gradient(to right, rgb(156, 112, 164), ...",False,Counterspell and Counterspell Ally reflect spe...,https://dota2protracker.com/static/facet_icons...,Magebane's Mirror,1,1,Anti-Mage,NaN,NaN,NaN,1
1,"linear-gradient(to right, rgb(148, 181, 186), ...",False,Anti-Mage gains damage when his enemies are at...,https://dota2protracker.com/static/facet_icons...,Mana Thirst,1,1,Anti-Mage,NaN,NaN,NaN,2
2,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Morphling's Primary Attribute is now <font col...,https://dota2protracker.com/static/facet_icons...,Ebb,10,10,Morphling,NaN,NaN,NaN,1
3,"linear-gradient(to right, rgb(159, 60, 60), rg...",False,Morphling's Primary Attribute is now <font col...,https://dota2protracker.com/static/facet_icons...,Flow,10,10,Morphling,NaN,NaN,NaN,2
4,"linear-gradient(to right, rgb(148, 181, 186), ...",False,Activate to temporarily apply a negative debuf...,https://dota2protracker.com/static/facet_icons...,Tag Team,100,100,Tusk,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
259,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Timber Chain splinters the tree into all direc...,https://dota2protracker.com/static/facet_icons...,Shredder,98,98,Timbersaw,NaN,NaN,NaN,1
260,"linear-gradient(to right, rgb(159, 60, 60), rg...",False,Launches a second independent Chakram that fol...,https://dota2protracker.com/static/facet_icons...,Twisted Chakram,98,98,Timbersaw,NaN,NaN,NaN,2
261,"linear-gradient(to right, rgb(202, 193, 148), ...",False,Warpath grants scaling Attack Speed instead of...,https://dota2protracker.com/static/facet_icons...,Berserk,99,99,Bristleback,NaN,NaN,NaN,1
262,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Bristleback now fires Viscous Nasal Goo instea...,https://dota2protracker.com/static/facet_icons...,Snot Rocket,99,99,Bristleback,NaN,NaN,NaN,2


In [14]:
def get_d2pt_page_table_facets(driver):
    time.sleep(0.2)
    # finding table rows in site and converting to format, readable by pandas
    category_name_elements = driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-3.gap-1"
    )
    category_name_elements += driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-4.gap-1"
    )
    hero_rows = [item.text.split("\n") for item in category_name_elements]
    hero_columns = [item for item in driver.find_element(
        By.CSS_SELECTOR, ".flex.gap-1.font-medium.text-sm.mb-1"
    ).text.split("\n") if item != "Trend"]
        
    # Dataframe dtype converts
    df_heroes_table = pd.DataFrame(data=hero_rows, columns=hero_columns)
    df_heroes_table = df_heroes_table.apply(pd.to_numeric, errors='ignore')
    df_heroes_table = df_heroes_table.round(1)
    return df_heroes_table

In [15]:
print("checking facets...")
print("fetching facets info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table_facets(driver)
df_1["Role"] = "pos 1"

print("fetching facets info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table_facets(driver)
df_2["Role"] = "pos 2"

print("fetching facets info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table_facets(driver)
df_3["Role"] = "pos 3"

print("fetching facets info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table_facets(driver)
df_4["Role"] = "pos 4"

print("fetching facets info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table_facets(driver)
df_5["Role"] = "pos 5"

df_full_facets = pd.concat([df_1,df_2,df_3,df_4,df_5], axis=0)

df_full_facets = df_full_facets.rename({
    "Hero": "hero",
    "Facet": "facet",
    "Matches": "Matches",
    "Win Rate": "Win Rate"
}, axis=1)[["hero", "facet", "Matches", "Win Rate", "Role"]]

df_full_facets["Win Rate"] = df_full_facets["Win Rate"].apply(lambda x: x.replace("%", ""))
df_full_facets = df_full_facets.apply(pd.to_numeric, errors='ignore')

df_full_facets

checking facets...
fetching facets info for pos 1
fetching facets info for pos 2
fetching facets info for pos 3
fetching facets info for pos 4
fetching facets info for pos 5


,hero,facet,Matches,Win Rate,Role
0,Bane,DREAM STALKER,6,50.0,pos 1
1,Mirana,SOLAR FLARE,20,25.0,pos 1
2,Lich,FROSTBOUND,10,20.0,pos 1
3,Dazzle,POISON BLOOM,8,37.5,pos 1
4,Ancient Apparition,BONE CHILL,3,0.0,pos 1
...,...,...,...,...,...
259,Necrophos,PROFANE POTENCY,4,25.0,pos 5
260,Wraith King,BONE GUARD,13,53.8,pos 5
261,Templar Assassin,VOIDBLADES,13,23.1,pos 5
262,Underlord,DEMON'S REACH,1,100.0,pos 5


## Results

In [86]:
print("creating in-game loadouts")

creating in-game loadouts


In [87]:
import pandas as pd


class HeroConfigProcessor:
    def __init__(self, df, name, data_type="facet"):
        """
        Класс для обработки и создания конфигураций на основе DataFrame.

        Args:
            df (pd.DataFrame): DataFrame с данными героев.
            name (str): Имя конфигурации.
            data_type (str): Тип данных ('facet' или 'regular').
        """
        self.df = df
        self.name = name
        self.data_type = data_type

    def get_hero_ids(
        self,
        position,
        facet_number=None,
        facet_id=None,
        wr_threshold=50,
        matches_threshold=50,
        rating_threshold=None,
        sort_by="Win Rate",
        ascending=False,
    ):
        """
        Получает список идентификаторов героев для указанной позиции.

        Args:
            position (int): Позиция героя (1 для pos 1, 2 для pos 2 и т.д.).
            facet_number (int, optional): Номер фасета.
            facet_id (int, optional): Идентификатор фасета (название фасета).
            wr_threshold (float, optional): Минимальный win rate.
            matches_threshold (int, optional): Минимальное количество матчей.
            rating_threshold (float, optional): Минимальный D2PT рейтинг.
            sort_by (str, optional): Поле для сортировки ('Win Rate', 'Matches', 'D2PT Rating').
            ascending (bool, optional): Порядок сортировки (по умолчанию убывающий).

        Returns:
            list: Список идентификаторов героев.
        """
        position_str = f"pos {position}"

        # Фильтруем данные по базовым критериям
        filtered_df = self.df[
            (self.df["Role"] == position_str)
            & (self.df["Matches"] > matches_threshold)
            & (self.df["Win Rate"] > wr_threshold)
        ]

        # Фильтрация по D2PT Rating, если указано
        if rating_threshold is not None and "D2PT Rating" in self.df.columns:
            filtered_df = filtered_df[filtered_df["D2PT Rating"] > rating_threshold]

        # Фильтрация для фасетов
        if self.data_type == "facet":
            if facet_number is not None:
                if facet_number == "3+":
                    filtered_df = filtered_df[filtered_df["facet_number"] > 2]
                else:
                    filtered_df = filtered_df[
                        filtered_df["facet_number"] == facet_number
                    ]
            if facet_id is not None:
                filtered_df = filtered_df[filtered_df["facet"] == facet_id]

        # Сортировка данных
        if sort_by in filtered_df.columns:
            filtered_df = filtered_df.sort_values(by=sort_by, ascending=ascending)

        return filtered_df["hero_id"].tolist()

    def build_config(
        self,
        wr_threshold=50,
        matches_threshold=50,
        rating_threshold=None,
        sort_by="Win Rate",
        ascending=False,
    ):
        """
        Создает конфигурацию на основе обработанных данных.

        Args:
            wr_threshold (float, optional): Минимальный win rate.
            matches_threshold (int, optional): Минимальное количество матчей.
            rating_threshold (float, optional): Минимальный D2PT рейтинг.
            sort_by (str, optional): Поле для сортировки ('Win Rate', 'Matches', 'D2PT Rating').
            ascending (bool, optional): Порядок сортировки (по умолчанию убывающий).

        Returns:
            dict: Конфигурация категорий.
        """
        if self.data_type == "facet":
            return self._build_facet_config(
                wr_threshold, matches_threshold, rating_threshold, sort_by, ascending
            )
        elif self.data_type == "regular":
            return self._build_regular_config(
                wr_threshold, matches_threshold, rating_threshold, sort_by, ascending
            )
        else:
            raise ValueError(f"Unsupported data type: {self.data_type}")

    def _build_facet_config(
        self, wr_threshold, matches_threshold, rating_threshold, sort_by, ascending
    ):
        """
        Создает конфигурацию для фасетов.

        Returns:
            dict: Конфигурация фасетов.
        """
        margin = 20
        max_height = 570
        max_width = 1180 - 100
        width = max_width / 5 - margin
        height = max_height / 2 - margin

        height_3 = (max_height / 5) - 20
        width_3 = 70

        categories = []
        for position in range(1, 6):
            hero_ids_facet_1 = self.get_hero_ids(
                position,
                facet_number=1,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )
            hero_ids_facet_2 = self.get_hero_ids(
                position,
                facet_number=2,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )
            hero_ids_facet_3_plus = self.get_hero_ids(
                position,
                facet_number="3+",
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )

            if hero_ids_facet_1:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 1",
                        "x_position": (width + margin) * (position - 1),
                        "y_position": 0,
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_facet_1,
                    }
                )
            if hero_ids_facet_2:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 2",
                        "x_position": (width + margin) * (position - 1),
                        "y_position": height + margin,
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_facet_2,
                    }
                )
            if hero_ids_facet_3_plus:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 3+",
                        "x_position": max_width + margin,
                        "y_position": (height_3 + margin) * (position - 1),
                        "width": width_3,
                        "height": height_3,
                        "hero_ids": hero_ids_facet_3_plus,
                    }
                )

        return {
            "config_name": self.name,
            "categories": categories,
        }

    def _build_regular_config(
        self, wr_threshold, matches_threshold, rating_threshold, sort_by, ascending
    ):
        """
        Создает конфигурацию для регулярного конфига.

        Returns:
            dict: Конфигурация регулярного конфига.
        """
        margin = 20
        width = 585 - margin
        height = 189 - margin

        included_hero_ids = set()
        categories = []

        for position in range(1, 6):
            hero_ids_regular = self.get_hero_ids(
                position,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )

            included_hero_ids.update(hero_ids_regular)

            if position <= 3:  # Верхние категории
                categories.append(
                    {
                        "category_name": f"Regular Pos {position}",
                        "x_position": 0.0,
                        "y_position": (height + margin) * (position - 1),
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_regular,
                    }
                )
            else:  # Нижние категории
                categories.append(
                    {
                        "category_name": f"Regular Pos {position}",
                        "x_position": width + margin,
                        "y_position": (height + margin) * (position - 4),
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_regular,
                    }
                )

        all_hero_ids = set(self.df["hero_id"].unique())
        not_included_hero_ids = list(all_hero_ids - included_hero_ids)

        if not_included_hero_ids:
            categories.append(
                {
                    "category_name": "Not-Included",
                    "x_position": width + margin,
                    "y_position": (height + margin) * 2,
                    "width": width,
                    "height": height,
                    "hero_ids": not_included_hero_ids,
                }
            )

        return {
            "config_name": self.name,
            "categories": categories,
        }

In [88]:
# Оставляем только уникальные значения в правом DataFrame
all_facets_df_unique = all_facets_df.drop_duplicates(subset=["localized_name"], keep="first")

# Выполняем объединение
df_final = pd.merge(
    df_full,
    all_facets_df_unique[["id", "localized_name"]],  # Оставляем только нужные столбцы
    left_on=["Hero"],  # Соответствие по названию фасета и герою
    right_on=["localized_name"],  # Название фасета и героя
    how="left"  # Оставляем все строки из df_full
)

# Переименовываем столбец "id" в "hero_id"
df_final.rename(columns={"id": "hero_id"}, inplace=True)

# Перемещаем "hero_id" на нулевую позицию
cols = ["hero_id"] + [col for col in df_final.columns if col != "hero_id"]
df_final = df_final[cols]

# Заменить hero_id на 76 для строк, где Hero = "Outworld Destroyer"
df_final.loc[df_final['Hero'] == "Outworld Destroyer", 'hero_id'] = 76

# Результат
display(df_final)


,hero_id,Rank,Role,Hero,D2PT Rating,Matches,Win Rate,WR 9500+ MMR,Contest Rate,Radiant,Dire,Expert,1st Phase,2nd Phase,Lastpick,Networth@10,localized_name
0,73,#1,pos 1,Alchemist,3479,4813,52.8,54.4% 2284 +1.6%,34.2,53.4,52.2,58.7% (825),49.8,54.1,51.2,"{'main': 5544, 'range_min': 4633, 'range_max':...",Alchemist
1,94,#2,pos 1,Medusa,3337,4864,52.4,51.7% 1945 -0.7%,47.3,54.8,49.9,59.4% (623),48.0,52.2,52.6,"{'main': 4577, 'range_min': 3892, 'range_max':...",Medusa
2,11,#3,pos 1,Shadow Fiend,3207,3953,51.6,52.9% 1313 +1.2%,72.5,54.0,49.1,56.9% (195),51.3,53.2,53.7,"{'main': 4713, 'range_min': 3864, 'range_max':...",Shadow Fiend
3,48,#4,pos 1,Luna,3136,6284,51.1,48.6% 2336 -2.5%,67.1,53.3,48.9,60.1% (634),51.7,50.8,58.7,"{'main': 4466, 'range_min': 3701, 'range_max':...",Luna
4,10,#5,pos 1,Morphling,3095,5204,51.4,51.3% 2158 -0.1%,40.9,53.4,49.4,54.7% (1164),47.4,51.9,53.2,"{'main': 4468, 'range_min': 3764, 'range_max':...",Morphling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,108,#122,pos 5,Underlord,0,22,50.0,100.0% 6 +50.0%,5.8,40.0,52.9,50.0% (16),47.4,0.0,0.0,"{'main': 3629, 'range_min': 2540, 'range_max':...",Underlord
626,120,#123,pos 5,Pangolier,0,29,44.8,50.0% 10 +5.2%,28.3,50.0,36.4,0.0% (0),25.0,54.5,0.0,"{'main': 2395, 'range_min': 1707, 'range_max':...",Pangolier
627,129,#124,pos 5,Mars,0,16,43.8,50.0% 4 +6.3%,22.7,50.0,40.0,0.0% (0),33.3,80.0,33.3,"{'main': 2302, 'range_min': 1387, 'range_max':...",Mars
628,126,#125,pos 5,Void Spirit,0,7,14.3,0.0% 2 -14.3%,7.3,0.0,50.0,0.0% (0),0.0,100.0,0.0,"{'main': 2668, 'range_min': 1411, 'range_max':...",Void Spirit


In [89]:
# Приводим названия фасетов к единому регистру
df_full_facets["facet_upper"] = df_full_facets["facet"].str.upper()  # В верхний регистр
all_facets_df["name_upper"] = all_facets_df["name"].str.upper()  # В верхний регистр

# Выполняем объединение (merge) двух DataFrame по колонкам с названиями фасетов
df_final_facets = pd.merge(
    df_full_facets,
    all_facets_df[["facet_number", "id", "name_upper", "localized_name"]],  # Оставляем только нужные столбцы
    left_on=["facet_upper", "hero"],  # Соответствие по названию фасета и герою
    right_on=["name_upper", "localized_name"],  # Название фасета и героя
    how="left"  # Оставляем все строки из df_full_facets
)

# Удаляем временные колонки с верхним регистром (если они не нужны)
df_final_facets.drop(columns=["facet_upper", "name_upper"], inplace=True)

# Переименовываем столбец "id" в "hero_id"
df_final_facets.rename(columns={"id": "hero_id"}, inplace=True)

# Перемещаем "hero_id" на нулевую позицию
cols = ["hero_id"] + [col for col in df_final_facets.columns if col != "hero_id"]
df_final_facets = df_final_facets[cols]

# Итоговые столбцы
df_final_facets = df_final_facets[["hero_id", "hero", "facet", "Matches", "Win Rate", "Role", "facet_number"]]

# Результат
display(df_final_facets)


,hero_id,hero,facet,Matches,Win Rate,Role,facet_number
0,3,Bane,DREAM STALKER,6,50.0,pos 1,1
1,9,Mirana,SOLAR FLARE,20,25.0,pos 1,2
2,31,Lich,FROSTBOUND,10,20.0,pos 1,1
3,50,Dazzle,POISON BLOOM,8,37.5,pos 1,2
4,68,Ancient Apparition,BONE CHILL,3,0.0,pos 1,1
...,...,...,...,...,...,...,...
1315,36,Necrophos,PROFANE POTENCY,4,25.0,pos 5,1
1316,42,Wraith King,BONE GUARD,13,53.8,pos 5,1
1317,46,Templar Assassin,VOIDBLADES,13,23.1,pos 5,1
1318,108,Underlord,DEMON'S REACH,1,100.0,pos 5,1


In [105]:
config = {
    "version": 3,
    "configs": [
        HeroConfigProcessor(df_final_facets, "Facet Matches>200", data_type="facet").build_config(matches_threshold=200, wr_threshold=51),
        HeroConfigProcessor(df_final_facets, "Facet Matches>50", data_type="facet").build_config(matches_threshold=50, wr_threshold=51),
        HeroConfigProcessor(df_final, "Win Rate", data_type="regular").build_config(matches_threshold=200, wr_threshold=51),
        HeroConfigProcessor(df_final, "D2PT", data_type="regular").build_config(matches_threshold=200, wr_threshold=0, sort_by="D2PT Rating"),
    ],
}

In [115]:
import threading


threading.Thread(target=lambda: driver.quit() if driver.service.process else None).start()

## Find Steam

In [106]:
import os
import winreg
import win32api

def read_reg(ep, p = r"", k = ''):
    try:
        key = winreg.OpenKeyEx(ep, p)
        value = winreg.QueryValueEx(key,k)
        if key:
            winreg.CloseKey(key)
        return value[0]
    except Exception as e:
        return None
    return None

Path1 = "{}\\Microsoft\\Windows\\Start Menu\\Programs\\Steam\\Steam.lnk".format(os.getenv('APPDATA'))
if os.path.exists(Path1):
    import sys
    import win32com.client 

    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(Path1)
    Path1Res = shortcut.Targetpath
else:
    Path1Res = False
Path2 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Wow6432Node\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
Path3 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
if not os.path.exists(Path2):
    Path2 = None
if not os.path.exists(Path3):
    Path3 = None
PossiblePaths = [r"X:\Steam\steam.exe", r"X:\Program Files\Steam\steam.exe", r"X:\Program Files (x86)\Steam\steam.exe"]
ValidHardPaths = []
for Drive in win32api.GetLogicalDriveStrings().split('\000')[:-1]:
    Drive = Drive.replace(':\\', '')
    for path in PossiblePaths:
        path = path.replace("X", Drive)
        if os.path.exists(path):
            ValidHardPaths.append(path)
if len(ValidHardPaths) == 0:
    ValidHardPaths = ["None"]
ValidHardPaths

['C:\\Program Files (x86)\\Steam\\steam.exe']

In [107]:
steam_dir = os.path.dirname(os.path.realpath(ValidHardPaths[0]))
steam_dir = os.path.join(steam_dir, "userdata")
steam_dir

'C:\\Program Files (x86)\\Steam\\userdata'

In [108]:
dirs_to_check = [os.path.join(steam_dir, id, "570", "remote", "cfg") for id in os.listdir(steam_dir)]
cfg_dirs = [dir for dir in dirs_to_check if os.path.isdir(dir)]
cfg_dirs

['C:\\Program Files (x86)\\Steam\\userdata\\106615943\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\106675276\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\114487406\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\383613970\\570\\remote\\cfg']

In [109]:
for dir in cfg_dirs:
    with open(os.path.join(dir, "hero_grid_config.json"), 'w+') as f:
        json.dump(dict(config), f)

## Autorun

In [110]:
import getpass
import os
import sys
USER_NAME = getpass.getuser()

In [111]:
from win32com.client import Dispatch

bat_path = rf'C:\Users\{USER_NAME}\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Startup\{os.path.basename(sys.executable)}.lnk'
target = sys.executable

shell = Dispatch('WScript.Shell')
shortcut = shell.CreateShortCut(bat_path)
shortcut.Targetpath = target
shortcut.save()

In [112]:
print("Done!")

for remaining in range(2, 0, -1):
    time.sleep(1)

Done!
